In [1]:
# 환경변수에서 API 키 가져오기
# urllib.parse는 url 구문분석을 해주는 것 (%어쩌고 하는 것을 나눠주기 위해서 사용)

import os
from urllib.parse import unquote

API_KEY = unquote(os.environ['ANIMAL_API_KEY'])
LOCATIONIQ_TOKEN = os.environ['LOCATIONIQ_TOKEN']

In [2]:
#라이브러리 사용
# requests 사용 - HTTP 요청을 주고 받을 수 있음

import requests
requests.__version__

'2.22.0'

In [3]:
# pandas 사용

import pandas as pd
pd.__version__

'0.25.0'

In [4]:
# matplotlib 사용

import matplotlib
import matplotlib.pyplot as plt

matplotlib.__version__

'3.1.0'

In [5]:
# matplotlib에 맑은 고딕 글꼴 적용하기
# 그냥 가져와서 적용하다보면 이해가는 부분

import matplotlib.font_manager as fm

for font in fm.fontManager.ttflist:
    if font.name in ['AppleGotic', 'Malgun Gothic']:
        plt.rcParams['font.family'] = font.name
        break   # for문 빠져나오기 위해서 사용

plt.rcParams['font.family']

['Malgun Gothic']

In [6]:
# folium 패키지 설치
!pip install folium

In [7]:
# folium 버전 확인
import folium
folium.__version__

'0.10.0'

In [8]:
# folium 라이브러리의 Map과 HeatMap 사용

from folium import Map
from folium.plugins import HeatMap

###  동물보호관리시스템 유기동물 조회 서비스 API 사용

In [9]:
# 동물보호관리시스템 유기동물 조회 서비스 API
# 원래 URL: http://openapi.animal.go.kr/openapi/service/rest/abandonmentPublicSrvc/abandonmentPublic?serviceKey=BWRDQsRPrjeh94ucQHEGPYK2xE7bGlTnLAdwI91aartteeP%2BFm%2FO1TmlX8uWMv0xRuF7t5yVk7tFdDsu2vxsCg%3D%3D&bgnde=20190901&endde=20190930&numOfRows=10wadl&type=xml
# 이렇게 긴 URL을 물음표(?)를 기준으로 두 부분으로 나눔
# 물음표(?) 앞부분
url = 'http://openapi.animal.go.kr/openapi/service/rest/abandonmentPublicSrvc/abandonmentPublic'

# 물음표(?) 뒷부분
payload = {
    'serviceKey' : API_KEY,
    'bgnde' : '20190901',
    'endde' : '20190930',
    'numOfRows' : 10000,
}
# API 호출
response = requests.get(url, params=payload)
print(response)

<Response [200]>


In [10]:
# Response의 Text 길이 확인
# response는 응답이고, response.text는 응답 받은 내용

len(response.text)

7836208

In [11]:
# XML 데이터 파싱
import xml.etree.ElementTree as ET
root = ET.fromstring(response.text)
root

# fromstring은 문자열 값으로 존재하는 경우 사용
# xml파일로 존재하는 경우는 parser를 사용

<Element 'response' at 0x0C2DBD50>

In [12]:
# 첫번째 내용을 얻고 다음으로 넘어갑니다.
item = next(root.iter('item'))

# Item에 어떤 내용이 있는지 확인합니다.
list(item)

[<Element 'age' at 0x0C2E4A20>,
 <Element 'careAddr' at 0x0C2E4A50>,
 <Element 'careNm' at 0x0C2E4A80>,
 <Element 'careTel' at 0x0C2E4AB0>,
 <Element 'chargeNm' at 0x0C2E4AE0>,
 <Element 'colorCd' at 0x0C2E4B10>,
 <Element 'desertionNo' at 0x0C2E4B70>,
 <Element 'filename' at 0x0C2E4BA0>,
 <Element 'happenDt' at 0x0C2E4BD0>,
 <Element 'happenPlace' at 0x0C2E4C00>,
 <Element 'kindCd' at 0x0C2E4C30>,
 <Element 'neuterYn' at 0x0C2E4C60>,
 <Element 'noticeEdt' at 0x0C2E4C90>,
 <Element 'noticeNo' at 0x0C2E4CC0>,
 <Element 'noticeSdt' at 0x0C2E4CF0>,
 <Element 'officetel' at 0x0C2E4D20>,
 <Element 'orgNm' at 0x0C2E4D50>,
 <Element 'popfile' at 0x0C2E4D80>,
 <Element 'processState' at 0x0C2E4DB0>,
 <Element 'sexCd' at 0x0C2E4DE0>,
 <Element 'specialMark' at 0x0C2E4E10>,
 <Element 'weight' at 0x0C2E4E40>]

In [13]:
# <kindCd> 항목을 확인
# 트리에서 바로 아래에 있는 것을 '자녀'라고 부름

child = item.find('kindCd')
print(child)

<Element 'kindCd' at 0x0C2E4C30>


In [14]:
# 자녀의 내용(text)을 확인
print(child.text)

[개] 믹스견


In [15]:
# child가 무엇이었는지 확인하는 법
# tag는 <kindCd> 이런 것을 의미?

print(child.tag)

kindCd


In [17]:
# item의 모든 자녀 확인
for child in item:
    print(child.tag, child.text)

age 2016(년생)
careAddr 경상남도 합천군 합천읍 옥산로 16 (합천읍, 까치빌라) 태민동물병원
careNm 태민동물병원
careTel 055-931-7975
chargeNm 박중언
colorCd 흰색
desertionNo 448548201900232
filename http://www.animal.go.kr/files/shelter/2019/09/201909260809844_s.jpg
happenDt 20190926
happenPlace 용주면 고품2구 부흥리
kindCd [개] 믹스견
neuterYn N
noticeEdt 20191007
noticeNo 경남-합천-2019-00284
noticeSdt 20190926
officetel 055-930-3562
orgNm 경상남도 합천군
popfile http://www.animal.go.kr/files/shelter/2019/09/201909260809844.jpg
processState 보호중
sexCd F
specialMark 활발하고 건강함
weight 10(Kg)


In [18]:
# 태그만 모아서 컬럼으로 활용하기
# 컬럼 목록 얻기
item = next(root.iter('item'))

columns = []
for child in item:
    columns.append(child.tag)

columns

['age',
 'careAddr',
 'careNm',
 'careTel',
 'chargeNm',
 'colorCd',
 'desertionNo',
 'filename',
 'happenDt',
 'happenPlace',
 'kindCd',
 'neuterYn',
 'noticeEdt',
 'noticeNo',
 'noticeSdt',
 'officetel',
 'orgNm',
 'popfile',
 'processState',
 'sexCd',
 'specialMark',
 'weight']

In [19]:
# XML 데이터를 pandas DataFrame으로 변환

# dictionary의 list를 준비 
rows = []

# item을 하나씩 rows에 넣기
for item in root.iter('item'):
    row = {}
    # item의 자녀를 하나씩 row에 넣기
    # 이런 코드가 나오면 앞에 나온 for는 잊고 그 후 나오는 것에 집중
    for child in list(item):
        # dictionary는 key와 value를 함께 써서 추가 또는 변경
        row[child.tag] = child.text
        # item의 자녀 내용이 들어간 row를 rows에 넣어주기
    rows.append(row)

# dictionary의 list는 pandas의 DataFrame으로 쉽게 변경 가능 
df = pd.DataFrame(rows, columns=columns)

In [20]:
df

,age,careAddr,careNm,careTel,chargeNm,colorCd,desertionNo,filename,happenDt,happenPlace,...,noticeEdt,noticeNo,noticeSdt,officetel,orgNm,popfile,processState,sexCd,specialMark,weight
0,2016(년생),"경상남도 합천군 합천읍 옥산로 16 (합천읍, 까치빌라) 태민동물병원",태민동물병원,055-931-7975,박중언,흰색,448548201900232,http://www.animal.go.kr/files/shelter/2019/09/...,20190926,용주면 고품2구 부흥리,...,20191007,경남-합천-2019-00284,20190926,055-930-3562,경상남도 합천군,http://www.animal.go.kr/files/shelter/2019/09/...,보호중,F,활발하고 건강함,10(Kg)
1,2017(년생),"경상남도 합천군 합천읍 옥산로 16 (합천읍, 까치빌라) 태민동물병원",태민동물병원,055-931-7975,박중언,"갈색,흰색",448548201900231,http://www.animal.go.kr/files/shelter/2019/09/...,20190926,용주면 고품2구 부흥리,...,20191007,경남-합천-2019-00283,20190926,055-930-3562,경상남도 합천군,http://www.animal.go.kr/files/shelter/2019/09/...,보호중,F,사람을 잘따르고 온순함,8(Kg)
2,2018(년생),경상남도 통영시 광도면 죽림3로 33 (광도면) 사랑애완동물병원,사랑애완동물병원,055-648-7582,박정재,연갈색/크림색,448533201900660,http://www.animal.go.kr/files/shelter/2019/09/...,20190926,한산도수산 근처,...,20191007,경남-통영-2019-00445,20190926,055-650-6252,경상남도 통영시,http://www.animal.go.kr/files/shelter/2019/09/...,보호중,M,"밧줄노끈 착용,굉장히 겁이 많음",7.7(Kg)
3,2019(년생),경상남도 통영시 광도면 죽림3로 33 (광도면) 사랑애완동물병원,사랑애완동물병원,055-648-7582,박정재,갈색/크림색/흰색,448533201900659,http://www.animal.go.kr/files/shelter/2019/09/...,20190926,도천동 257-1,...,20191007,경남-통영-2019-00444,20190926,055-650-6252,경상남도 통영시,http://www.animal.go.kr/files/shelter/2019/09/...,보호중,M,"생후 7일령의 너무어린 자묘,출산한 모견과 합사/케어 중",0.09(Kg)
4,2019(년생),"경상남도 창원시 진해구 웅천로 218 (성내동, 농업기술센터) 205번지",진해유기동물보호소,055-225-5483,진해구,흰색,448532201900789,http://www.animal.go.kr/files/shelter/2019/09/...,20190926,진해유기동물보호소,...,20191007,경남-창원3-2019-00458,20190926,055-225-5483,경상남도 창원시 진해구,http://www.animal.go.kr/files/shelter/2019/09/...,보호중,M,"1개월, 남매들이 다 닮음, 사람손길을 좋아하는 아이.",1.3(Kg)
5,2019(년생),"경상남도 창원시 진해구 웅천로 218 (성내동, 농업기술센터) 205번지",진해유기동물보호소,055-225-5483,진해구,흰,448532201900788,http://www.animal.go.kr/files/shelter/2019/09/...,20190926,진해유기동물보호소,...,20191007,경남-창원3-2019-00457,20190926,055-225-5483,경상남도 창원시 진해구,http://www.animal.go.kr/files/shelter/2019/09/...,보호중,F,"1개월, 사랑스러움을 타고난 깜찍한 여자아이, 어린강아지들이 너무 많습니다. 눈여겨...",1.3(Kg)
6,2019(년생),"경상남도 창원시 진해구 웅천로 218 (성내동, 농업기술센터) 205번지",진해유기동물보호소,055-225-5483,진해구,갈색,448532201900787,http://www.animal.go.kr/files/shelter/2019/09/...,20190926,진해유기동물보호소,...,20191007,경남-창원3-2019-00456,20190926,055-225-5483,경상남도 창원시 진해구,http://www.animal.go.kr/files/shelter/2019/09/...,보호중,F,"보호소에서 태어난아이,1개월,유일하게 엄마를 닮은 숙녀, 태평한 성격,",1.3(Kg)
7,2019(년생),"경상남도 창원시 진해구 웅천로 218 (성내동, 농업기술센터) 205번지",진해유기동물보호소,055-225-5483,진해구,흰,448532201900786,http://www.animal.go.kr/files/shelter/2019/09/...,20190926,진해유기동물보호소,...,20191007,경남-창원3-2019-00455,20190926,055-225-5483,경상남도 창원시 진해구,http://www.animal.go.kr/files/shelter/2019/09/...,보호중,M,"1개월 1키로에 작은 아이, 순한얼굴, 검정콩 3개, 분홍빛 발가락.",1.0(Kg)
8,2019(년생),"경상남도 창원시 진해구 웅천로 218 (성내동, 농업기술센터) 205번지",진해유기동물보호소,055-225-5483,진해구,흰,448532201900785,http://www.animal.go.kr/files/shelter/2019/09/...,20190926,진해 유기동물보호소,...,20191007,경남-창원3-2019-00454,20190926,055-225-5483,경상남도 창원시 진해구,http://www.animal.go.kr/files/shelter/2019/09/...,보호중,M,"1개월, 보호소에서 태어난 아이,동글동글 귀여운 외모,",1.3(Kg)
9,2017(년생),"경상남도 창원시 진해구 웅천로 218 (성내동, 농업기술센터) 205번지",진해유기동물보호소,055-225-5483,진해구,흰,448532201900784,http://www.animal.go.kr/files/shelter/2019/09/...,20190926,진해구 웅동로 107번길 51,...,20191007,경남-창원3-2019-00453,20190926,055-225-5483,경상남도 창원시 진해구,http://www.animal.go.kr/files/shelter/2019/09/...,보호중,F,"칩x,진도믹스추정,마르고 배가많이 불러있음, 지치고 힘들어보임.",12.5(Kg)


In [21]:
# 1. 컬럼 목록
df.columns

Index(['age', 'careAddr', 'careNm', 'careTel', 'chargeNm', 'colorCd',
       'desertionNo', 'filename', 'happenDt', 'happenPlace', 'kindCd',
       'neuterYn', 'noticeEdt', 'noticeNo', 'noticeSdt', 'officetel', 'orgNm',
       'popfile', 'processState', 'sexCd', 'specialMark', 'weight'],
      dtype='object')

In [22]:
# 2. 전체 갯수와 컬럼 갯수 확인
df.shape

(10000, 22)

In [23]:
# 3. 데이터 일부 확인
df.head() 
# head()에 숫자 입력해주지 않을 경우 일반적으로 5개 정도 보여줌

,age,careAddr,careNm,careTel,chargeNm,colorCd,desertionNo,filename,happenDt,happenPlace,...,noticeEdt,noticeNo,noticeSdt,officetel,orgNm,popfile,processState,sexCd,specialMark,weight
0,2016(년생),"경상남도 합천군 합천읍 옥산로 16 (합천읍, 까치빌라) 태민동물병원",태민동물병원,055-931-7975,박중언,흰색,448548201900232,http://www.animal.go.kr/files/shelter/2019/09/...,20190926,용주면 고품2구 부흥리,...,20191007,경남-합천-2019-00284,20190926,055-930-3562,경상남도 합천군,http://www.animal.go.kr/files/shelter/2019/09/...,보호중,F,활발하고 건강함,10(Kg)
1,2017(년생),"경상남도 합천군 합천읍 옥산로 16 (합천읍, 까치빌라) 태민동물병원",태민동물병원,055-931-7975,박중언,"갈색,흰색",448548201900231,http://www.animal.go.kr/files/shelter/2019/09/...,20190926,용주면 고품2구 부흥리,...,20191007,경남-합천-2019-00283,20190926,055-930-3562,경상남도 합천군,http://www.animal.go.kr/files/shelter/2019/09/...,보호중,F,사람을 잘따르고 온순함,8(Kg)
2,2018(년생),경상남도 통영시 광도면 죽림3로 33 (광도면) 사랑애완동물병원,사랑애완동물병원,055-648-7582,박정재,연갈색/크림색,448533201900660,http://www.animal.go.kr/files/shelter/2019/09/...,20190926,한산도수산 근처,...,20191007,경남-통영-2019-00445,20190926,055-650-6252,경상남도 통영시,http://www.animal.go.kr/files/shelter/2019/09/...,보호중,M,"밧줄노끈 착용,굉장히 겁이 많음",7.7(Kg)
3,2019(년생),경상남도 통영시 광도면 죽림3로 33 (광도면) 사랑애완동물병원,사랑애완동물병원,055-648-7582,박정재,갈색/크림색/흰색,448533201900659,http://www.animal.go.kr/files/shelter/2019/09/...,20190926,도천동 257-1,...,20191007,경남-통영-2019-00444,20190926,055-650-6252,경상남도 통영시,http://www.animal.go.kr/files/shelter/2019/09/...,보호중,M,"생후 7일령의 너무어린 자묘,출산한 모견과 합사/케어 중",0.09(Kg)
4,2019(년생),"경상남도 창원시 진해구 웅천로 218 (성내동, 농업기술센터) 205번지",진해유기동물보호소,055-225-5483,진해구,흰색,448532201900789,http://www.animal.go.kr/files/shelter/2019/09/...,20190926,진해유기동물보호소,...,20191007,경남-창원3-2019-00458,20190926,055-225-5483,경상남도 창원시 진해구,http://www.animal.go.kr/files/shelter/2019/09/...,보호중,M,"1개월, 남매들이 다 닮음, 사람손길을 좋아하는 아이.",1.3(Kg)


In [26]:
# 컬럼별 결측값 갯수 확인
# isnull()은 null값이 존재하면 true 아닐경우는 False
data = df.isnull().sum() 
data

age               0
careAddr          0
careNm            0
careTel           0
chargeNm        197
colorCd           0
desertionNo       0
filename          0
happenDt          0
happenPlace       0
kindCd            0
neuterYn          0
noticeEdt         0
noticeNo          0
noticeSdt         0
officetel       606
orgNm             0
popfile           0
processState      0
sexCd             0
specialMark       0
weight            0
dtype: int64

In [ ]:
# 안 쓰는 컬럼 이름을 넣어 drop해서 simple 데이터 만들기
simple_df = df.drop(columns=[
    
])

# 정리된 데이터 확인
simple_df.head()